### Superstore DataSet ###

Contexto

Con las crecientes demandas y las feroces competencias en el mercado, un gigante de las supertiendas busca su conocimiento para comprender qué funciona mejor para ellos. Les gustaría saber a qué productos, regiones, categorías y segmentos de clientes deben dirigirse o evitar.

Incluso puede ir un paso más allá e intentar crear un modelo de regresión para predecir las ventas o los beneficios.


In [1]:
#Librerías básicas
import pandas as pd
import numpy as np 

#Librerías de Visualización
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Imputación de nulos
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer


pd.set_option('display.max_columns', None)

In [2]:
superstore = pd.read_csv(r"C:\Users\maria\Desktop\Sample - Superstore.csv", encoding='windows-1252')


In [3]:
superstore.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
superstore.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [5]:
superstore.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [6]:
# Renombramos columnas, unicamos en minúscula y sin espacios para trabajar con uniformidad.
nombresColumnas = superstore.columns
renombre_columnas = {columna: columna.lower().replace(' ', '_') for columna in nombresColumnas}
renombre_columnas


{'Row ID': 'row_id',
 'Order ID': 'order_id',
 'Order Date': 'order_date',
 'Ship Date': 'ship_date',
 'Ship Mode': 'ship_mode',
 'Customer ID': 'customer_id',
 'Customer Name': 'customer_name',
 'Segment': 'segment',
 'Country': 'country',
 'City': 'city',
 'State': 'state',
 'Postal Code': 'postal_code',
 'Region': 'region',
 'Product ID': 'product_id',
 'Category': 'category',
 'Sub-Category': 'sub-category',
 'Product Name': 'product_name',
 'Sales': 'sales',
 'Quantity': 'quantity',
 'Discount': 'discount',
 'Profit': 'profit'}

In [7]:
superstore.rename(columns= renombre_columnas, inplace=True)
superstore.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='object')

### Descripción de campos

- **row_id**: Identificador único de la fila. Utilizado para distinguir cada fila del dataset.
  
- **order_id**: Identificador único del pedido. Cada pedido tiene su propio ID para facilitar el seguimiento.
  
- **order_date**: Fecha en que se realizó el pedido.
  
- **ship_date**: Fecha en que el pedido fue enviado.('Second Class', 'Standard Class', 'First Class', 'Same Day')
  
- **ship_mode**: Modo de envío del pedido (por ejemplo, envío estándar, urgente, etc.).
  
- **customer_id**: Identificador único del cliente que realizó el pedido.
  
- **customer_name**: Nombre del cliente que realizó el pedido.
  
- **segment**: Segmento de mercado al que pertenece el cliente ('Consumer', 'Corporate', 'Home Office').
  
- **country**: País desde donde se realizó el pedido.
  
- **city**: Ciudad desde donde se realizó el pedido.
  
- **state**: Estado o provincia desde donde se realizó el pedido.
  
- **postal_code**: Código postal del cliente o de la ubicación del envío.
  
- **region**: Región geográfica o de mercado donde se realizó el pedido.
  
- **product_id**: Identificador único del producto en el pedido.
  
- **category**: Categoría a la que pertenece el producto ('Furniture', 'Office Supplies', 'Technology').
  
- **sub-category**: Subcategoría dentro de la categoría principal del producto. ('Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage', 'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper','Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines','Copiers')
  
- **product_name**: Nombre del producto.
  
- **sales**: Monto de las ventas generadas por el pedido.
  
- **quantity**: Cantidad de productos pedidos.
  
- **discount**: Descuento aplicado en el pedido (generalmente expresado como porcentaje o monto).
  
- **profit**: Ganancia obtenida por el pedido, calculada después de aplicar los costos y descuentos.


In [8]:
superstore.shape

(9994, 21)

In [9]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   row_id         9994 non-null   int64  
 1   order_id       9994 non-null   object 
 2   order_date     9994 non-null   object 
 3   ship_date      9994 non-null   object 
 4   ship_mode      9994 non-null   object 
 5   customer_id    9994 non-null   object 
 6   customer_name  9994 non-null   object 
 7   segment        9994 non-null   object 
 8   country        9994 non-null   object 
 9   city           9994 non-null   object 
 10  state          9994 non-null   object 
 11  postal_code    9994 non-null   int64  
 12  region         9994 non-null   object 
 13  product_id     9994 non-null   object 
 14  category       9994 non-null   object 
 15  sub-category   9994 non-null   object 
 16  product_name   9994 non-null   object 
 17  sales          9994 non-null   float64
 18  quantity

In [10]:
superstore.describe().T

,count,mean,std,min,25%,50%,75%,max
row_id,9994.0,4997.500000,2885.163629,1.000,2499.25000,4997.5000,7495.750,9994.000
postal_code,9994.0,55190.379428,32063.693350,1040.000,23223.00000,56430.5000,90008.000,99301.000
sales,9994.0,229.858001,623.245101,0.444,17.28000,54.4900,209.940,22638.480
quantity,9994.0,3.789574,2.225110,1.000,2.00000,3.0000,5.000,14.000
discount,9994.0,0.156203,0.206452,0.000,0.00000,0.2000,0.200,0.800
profit,9994.0,28.656896,234.260108,-6599.978,1.72875,8.6665,29.364,8399.976


In [11]:
superstore.describe(include='O').T 

,count,unique,top,freq
order_id,9994,5009,CA-2017-100111,14
order_date,9994,1237,9/5/2016,38
ship_date,9994,1334,12/16/2015,35
ship_mode,9994,4,Standard Class,5968
customer_id,9994,793,WB-21850,37
customer_name,9994,793,William Brown,37
segment,9994,3,Consumer,5191
country,9994,1,United States,9994
city,9994,531,New York City,915
state,9994,49,California,2001


Se ajustan las columnas con referencia fechas para que tengas si clasitficacion correcta y poder trabajar mejor con las líneas de tiempo. Se crearan dos columans nuevas  de año y mes con la finalidad de hacer mejores análisi.

In [12]:
fecha = superstore['order_date'].str.split("/", expand=True)

In [13]:
superstore['contact_year'] = fecha[2]
superstore['contact_month'] = fecha[1]

In [14]:
superstore['ship_date'].replace('/', '-', regex=True, inplace = True)

C:\Users\maria\AppData\Local\Temp\ipykernel_13796\3548228920.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  superstore['ship_date'].replace('/', '-', regex=True, inplace = True)


In [15]:
superstore['order_date'].replace('/', '-',inplace = True)


In [16]:
superstore.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,contact_year,contact_month
0,1,CA-2016-152156,11/8/2016,11-11-2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016,8
1,2,CA-2016-152156,11/8/2016,11-11-2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016,8
2,3,CA-2016-138688,6/12/2016,6-16-2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016,12
3,4,US-2015-108966,10/11/2015,10-18-2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015,11
4,5,US-2015-108966,10/11/2015,10-18-2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015,11


In [17]:
superstore["order_date"]= pd.to_datetime(superstore["order_date"], format='mixed')

In [18]:
superstore["ship_date"]= pd.to_datetime(superstore["order_date"], format='mixed')

In [19]:
superstore["contact_year"]= superstore["contact_year"].astype(int)

In [20]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   row_id         9994 non-null   int64         
 1   order_id       9994 non-null   object        
 2   order_date     9994 non-null   datetime64[ns]
 3   ship_date      9994 non-null   datetime64[ns]
 4   ship_mode      9994 non-null   object        
 5   customer_id    9994 non-null   object        
 6   customer_name  9994 non-null   object        
 7   segment        9994 non-null   object        
 8   country        9994 non-null   object        
 9   city           9994 non-null   object        
 10  state          9994 non-null   object        
 11  postal_code    9994 non-null   int64         
 12  region         9994 non-null   object        
 13  product_id     9994 non-null   object        
 14  category       9994 non-null   object        
 15  sub-category   9994 n

In [21]:
# Se realiza la extración de los únicos en lso campos que se han considerando importantes y controlables, para ayudar en el futuro análisis
lista_unicos = ['segment','ship_mode','category', 'sub-category']
for item in lista_unicos:
  print(f'Los valores únicos del campo {item} son:')
  print (superstore[item].unique())
  print()
  


Los valores únicos del campo segment son:
['Consumer' 'Corporate' 'Home Office']

Los valores únicos del campo ship_mode son:
['Second Class' 'Standard Class' 'First Class' 'Same Day']

Los valores únicos del campo category son:
['Furniture' 'Office Supplies' 'Technology']

Los valores únicos del campo sub-category son:
['Bookcases' 'Chairs' 'Labels' 'Tables' 'Storage' 'Furnishings' 'Art'
 'Phones' 'Binders' 'Appliances' 'Paper' 'Accessories' 'Envelopes'
 'Fasteners' 'Supplies' 'Machines' 'Copiers']



Finaliza la preparación de los datos para poder estudiar las variable y mértricas.

In [22]:
superstore.to_csv(r'superstore_limpio.csv')